## [Simple small-field polynomial commitment scheme](https://eprint.iacr.org/2023/1784.pdf#page=12.15)
Define $\Pi=$ (Setup, Commit, Open, Prove, Verify):
- params $\leftarrow \Pi$.Setup $\left(1^\lambda, \ell, K\right)$. On input $1^\lambda, \ell$, and $K$, choose integers $\ell_0$ and $\ell_1$ for which $\ell_0+\ell_1=\ell$, and write $m_0:=2^{\ell_0}$ and $m_1:=2^{\ell_1}$. Return an extension field $L / K$ for which $|L| \geq 2^{\omega(\log \lambda)}$, an $\left[n, m_1, d\right]$-code $C \subset K^n$ for which $n=2^{O(\ell)}$ and $d=\Omega(n)$, and a repetition parameter $\gamma=\Theta(\lambda)$.
- $(c, u) \leftarrow \Pi$.Commit(params, $t)$. On input $t\left(X_0, \ldots, X_{\ell-1}\right) \in K\left[X_0, \ldots, X_{\ell-1}\right]^{\preceq 1}$, express $t=$ $\left(t_0, \ldots, t_{2^{\ell}-1}\right)$ in coordinates with respect to the Lagrange basis on $\{0,1\}^{\ell}$, collate the resulting vector into an $m_0 \times m_1$ matrix $\left(t_i\right)_{i=0}^{m_0-1}$, and encode $\left(t_i\right)_{i=0}^{m_0-1}$ row-wise, so obtaining a further matrix $\left(u_i\right)_{i=0}^{m_0-1}$. Output a Merkle commitment $c$ to $\left(u_i\right)_{i=0}^{m_0-1}$ and the opening hint $u:=\left(u_i\right)_{i=0}^{m_0-1}$.
- $b \leftarrow \Pi$.Open(params, $c ; t, u)$. On input the root $c$, opening $t\left(X_0, \ldots, X_{\ell-1}\right) \in K\left[X_0, \ldots, X_{\ell-1}\right]^{\preceq 1}$, and opening hint a set of distinct Merkle paths against $c$, missing the columns $M \subset\{0, \ldots, n-1\}$, say, write $t$ into a matrix $\left(t_i\right)_{i=0}^{m_0-1}$ and check $\left|\Delta^{m_0}\left(\left(u_i\right)_{i=0}^{m_0-1},\left(\operatorname{Enc}\left(t_i\right)\right)_{i=0}^{m_0-1}\right) \cup M\right| \stackrel{?}{<} \frac{d}{2}$.

Define $\Pi$.Prove and $\Pi$.Verify by applying the Fiat-Shamir heuristic to the following interactive protocol, where $\mathcal{P}$ has $t\left(X_0, \ldots, X_{\ell-1}\right)$ and $\left(u_i\right)_{i=0}^{m_0-1}$, and $\mathcal{P}$ and $\mathcal{V}$ have $c, s \in L$, and $\left(r_0, \ldots, r_{\ell-1}\right) \in L^{\ell}$.

- $\mathcal{P}$ sends $\mathcal{V}$ the matrix-vector product $t^{\prime}:=\bigotimes_{i=\ell_1}^{\ell-1}\left(1-r_i, r_i\right) \cdot\left(t_i\right)_{i=0}^{m_0-1}$ in the clear.
- For each $i \in\{0, \ldots, \gamma-1\}, \mathcal{V}$ samples $j_i \leftarrow\{0, \ldots, n-1\}$. $\mathcal{V}$ sends $\mathcal{P}$ the set $J:=\left\{j_0, \ldots, j_{\gamma-1}\right\}$.
- $\mathcal{P}$ sends $\mathcal{V}$ the columns $\left\{\left(u_{i, j}\right)_{i=0}^{m_0-1}\right\}_{j \in J}$, each featuring an accompanying Merkle path against $c$.
- $\mathcal{V}$ computes $\widehat{\operatorname{Enc}}\left(t^{\prime}\right)$. For each $j \in J, \mathcal{V}$ verifies the Merkle path attesting to $\left(u_{i, j}\right)_{i=0}^{m_0-1}$, and moreover checks $\bigotimes_{i=\ell_1}^{\ell-1}\left(1-r_i, r_i\right) \cdot\left(u_{i, j}\right)_{i=0}^{m_0-1} \stackrel{?}{=} \widehat{\operatorname{Enc}}\left(t^{\prime}\right)_j$. Finally, $\mathcal{V}$ requires $s \stackrel{?}{=} t^{\prime} \cdot \bigotimes_{i=0}^{\ell_1-1}\left(1-r_i, r_i\right)$

In [1]:
def simple_binius_proof(evaluations, evaluation_point):
    cls, evaluations, evaluation_point = \
        enforce_type_compatibility(evaluations, evaluation_point)

    # Rearrange evaluations into a row_length * row_count grid
    log_row_length, log_row_count, row_length, row_count = \
        choose_row_length_and_count(log2(len(evaluations)))
    rows = [
        evaluations[i:i+row_length]
        for i in range(0, len(evaluations), row_length)
    ]

    # Extend each row using a Reed-Solomon code
    extended_rows = [extend(row, EXPANSION_FACTOR) for row in rows]
    extended_row_length = row_length * EXPANSION_FACTOR

    # Compute t_prime, a linear combination of the rows
    # The linear combination is carefully chosen so that the evaluation of the
    # multilinear polynomial at the `evaluation_point` is itself either on
    # t_prime, or a linear combination of elements of t_prime
    row_combination = \
        evaluation_tensor_product(evaluation_point[log_row_length:])
    assert len(row_combination) == len(rows) == row_count
    t_prime = [
        sum([rows[i][j] * row_combination[i] for i in range(row_count)], cls(0))
        for j in range(row_length)
    ]

    # Pack columns into a Merkle tree, to commit to them
    columns = [
        [row[j] for row in extended_rows]
        for j in range(extended_row_length)
    ]
    packed_columns = [
        b''.join(x.to_bytes(BYTES_PER_ELEMENT, 'little') for x in col)
        for col in columns
    ]
    merkle_tree = merkelize(packed_columns)
    root = get_root(merkle_tree)

    # Challenge in a few positions, to get branches
    challenges = [
        int.from_bytes(hash(root + bytes([i])), 'little') % extended_row_length
        for i in range(NUM_CHALLENGES)
    ]
    return {
        'root': root,
        'evaluation_point': evaluation_point,
        'eval': multilinear_poly_eval(evaluations, evaluation_point),
        't_prime': t_prime,
        'columns': [columns[c] for c in challenges],
        'branches': [get_branch(merkle_tree, c) for c in challenges],
    }


In [2]:
def verify_simple_binius_proof(proof):
    cls, columns, evaluation_point, value, t_prime = enforce_type_compatibility(
        proof['columns'],
        proof['evaluation_point'],
        proof['eval'],
        proof['t_prime']
    )
    root, branches = proof["root"], proof["branches"]

    # Compute the row length and row count of the grid. Should output same
    # numbers as what prover gave
    log_row_length, log_row_count, row_length, row_count = \
        choose_row_length_and_count(len(evaluation_point))
    extended_row_length = row_length * EXPANSION_FACTOR

    # Compute challenges. Should output the same as what prover computed
    challenges = [
        int.from_bytes(hash(root + bytes([i])), 'little') % extended_row_length
        for i in range(NUM_CHALLENGES)
    ]

    # Verify the correctness of the Merkle branches
    for challenge, branch, column in zip(challenges, branches, columns):
        packed_column = \
            b''.join(x.to_bytes(BYTES_PER_ELEMENT, 'little') for x in column)
        print(f"Verifying Merkle branch for column {challenge}")
        assert verify_branch(root, challenge, packed_column, branch)

    # Use the same Reed-Solomon code that the prover used to extend the rows,
    # but to extend t_prime
    extended_t_prime = extend(proof["t_prime"], EXPANSION_FACTOR)

    # Here, we take advantage of the linearity of the code. A linear combination
    # of the Reed-Solomon extension gives the same result as an extension of the
    # linear combination.
    row_combination = \
        evaluation_tensor_product(evaluation_point[log_row_length:])
    for column, challenge in zip(proof['columns'], challenges):
        expected_tprime = sum(
            [column[i] * row_combination[i] for i in range(row_count)],
            cls(0)
        )
        print(
            f"Testing challenge on column {challenge}: expected "
            f"{expected_tprime} computed {extended_t_prime[challenge]}"
        )
        assert expected_tprime == extended_t_prime[challenge]

    # Take the right linear combination of elements *within* t_prime to
    # extract the evaluation of the original multilinear polynomial at
    # the desired point
    col_combination = \
        evaluation_tensor_product(evaluation_point[:log_row_length])
    computed_eval = sum(
        [t_prime[i] * col_combination[i] for i in range(row_length)],
        cls(0)
    )
    print(f"Testing evaluation: expected {value} computed {computed_eval}")
    assert computed_eval == value
    return True

In [3]:
def choose_row_length_and_count(log_evaluation_count):
    log_row_length = log_evaluation_count // 2
    log_row_count = (log_evaluation_count + 1) // 2
    row_length = 1 << log_row_length
    row_count = 1 << log_row_count
    return log_row_length, log_row_count, row_length, row_count